In [1]:
import pdfplumber
import re
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
pdf = pdfplumber.open("Magnolia-Warehouse.pdf")
all_pages = pdf.pages
cleaned_lines = []
cleaned_list_of_dicts = []
for each_page in all_pages:
    text_of_each_page = each_page.extract_text()
    all_lines = text_of_each_page.splitlines()
    for each_line in all_lines:
        if each_line == "Address":
            continue
        elif each_line == "Owner Card  Mailing Label  Map and Details Report":
            continue
        elif each_line == "KGIS Parcel Report":
            continue
        else:
            cleaned_lines.append(each_line)
each_dict = {}
for index, cleaned_line in enumerate(cleaned_lines):
    if re.match(r"^09[0-9A-Z]{5,}$", cleaned_line):
        # parcel_id
        each_dict["parcel_id"] = cleaned_line
        each_dict["name"] = cleaned_lines[index + 1]
        cleaned_list_of_dicts.append(each_dict)
        each_dict = {}
    else:
        continue

In [3]:
len(cleaned_list_of_dicts)

192

In [4]:
# cleaned_list_of_dicts

In [10]:
driver = webdriver.Chrome(ChromeDriverManager().install())

/var/folders/2g/fbs070qj3qd3kw4ntm252trc0000gp/T/ipykernel_85574/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [11]:
def get_all_data(parcel_number, agree):
    driver.get('https://propertyinfo.knoxcountytn.gov/search/commonsearch.aspx?mode=realprop')
    if agree:
        driver.find_element(By.ID, "btAgree").click()
    driver.find_element(By.ID, "inpParid").send_keys(parcel_number)
    driver.find_element(By.ID, "btSearch").click()
    error_passing = driver.find_elements(By.ID, "searchResults")
    # print(error_passing)
    if len(error_passing) > 0:
        error_passing[0].find_element(By.CLASS_NAME, "SearchResults").click()
    driver.find_element(By.XPATH, '//*[@id="sidemenu"]/ul/li[3]/a').click()
    parcel_id_dirty = driver.find_element(By.CLASS_NAME, "DataletHeaderTop").text
    address_data = driver.find_elements(By.CLASS_NAME, "DataletHeaderBottom")
    parcel_id = parcel_id_dirty.split(": ")[1]
    sales_data = []
    try:
        sales_details = driver.find_element(By.ID, "Sales Summary")
        rows = sales_details.find_elements(By.TAG_NAME, "tr")[1:]
        for row in rows:
            all_data = row.find_elements(By.TAG_NAME, "td")
            if len(all_data) < 5:
                print("insufficient data")
                continue
            else:
                data = {}
                date = all_data[0].text
                book = all_data[1].text
                page = all_data[2].text
                price = all_data[3].text
                grantee = all_data[-1].text
                address = address_data[-1].text
                data["date"] = date
                data["book_number"] = book
                data["page"] = page
                data["sale_price"] = price
                data["grantee_name"] = grantee
                data["parcel_id_new"] = parcel_id
                data["address"] = address
                sales_data.append(data)
    except:
        data = {}
        data["date"] = "Not found"
        data["book_number"] = "Not found"
        data["page"] = "Not found"
        data["sale_price"] = "Not found"
        data["grantee_name"] = "Not found"
        data["parcel_id_new"] = "Not found"
        data["address"] = "Not found"
        sales_data.append(data)
    return(sales_data)

In [7]:
all_sales_data = []

In [12]:
agree = True
for index, each_parcel_dict in enumerate(cleaned_list_of_dicts[100:]):
    sales_data = get_all_data(each_parcel_dict["parcel_id"], agree)
    agree = False
    all_sales_data.extend(sales_data)
    print(index, "done")

insufficient data
0 done
insufficient data
1 done
insufficient data
2 done
insufficient data
3 done
insufficient data
4 done
insufficient data
5 done
insufficient data
6 done
insufficient data
7 done
insufficient data
8 done
insufficient data
9 done
insufficient data
10 done
insufficient data
11 done
insufficient data
12 done
insufficient data
13 done
insufficient data
14 done
insufficient data
15 done
insufficient data
16 done
insufficient data
17 done
insufficient data
18 done
insufficient data
19 done
insufficient data
20 done
insufficient data
21 done
insufficient data
22 done
insufficient data
23 done
insufficient data
24 done
insufficient data
25 done
insufficient data
26 done
insufficient data
27 done
insufficient data
28 done
insufficient data
29 done
insufficient data
30 done
insufficient data
31 done
insufficient data
32 done
insufficient data
33 done
insufficient data
34 done
insufficient data
35 done
insufficient data
36 done
insufficient data
37 done
insufficient data
38 d

In [9]:
# all_sales_data

In [14]:
all_sales_data

[{'date': '01/01/1962',
  'book_number': '00001193',
  'page': '0000570',
  'sale_price': '$0',
  'grantee_name': 'MONDAY JAMES STEPHENSON',
  'parcel_id_new': '095AC015'},
 {'date': '11/08/1976',
  'book_number': '00001595',
  'page': '0000371',
  'sale_price': '$0',
  'grantee_name': 'KNOXVILLES COMMUNITY DEV CORP',
  'parcel_id_new': '095AD00502'},
 {'date': '01/02/2020',
  'book_number': '20200324',
  'page': '0063309',
  'sale_price': '$0',
  'grantee_name': 'SEMPER FI PROPERTIES',
  'parcel_id_new': '095AH01401'},
 {'date': '12/31/2019',
  'book_number': '20210629',
  'page': '0107109',
  'sale_price': '$0',
  'grantee_name': 'MILLENNIUM TRUST COMPANY LLC',
  'parcel_id_new': '095AH01401'},
 {'date': '12/31/2019',
  'book_number': '20210623',
  'page': '0105521',
  'sale_price': '$0',
  'grantee_name': 'MILLENNIUM TRUST COMPANY LLC',
  'parcel_id_new': '095AH01401'},
 {'date': '08/28/2012',
  'book_number': '20120831',
  'page': '0014409',
  'sale_price': '$0',
  'grantee_name': 

In [15]:
pd.set_option('display.max_columns', None)
magnolia_df = pd.DataFrame(all_sales_data)

In [16]:
magnolia_df

,date,book_number,page,sale_price,grantee_name,parcel_id_new,address
0,01/01/1962,00001193,0000570,$0,MONDAY JAMES STEPHENSON,095AC015,NaN
1,11/08/1976,00001595,0000371,$0,KNOXVILLES COMMUNITY DEV CORP,095AD00502,NaN
2,01/02/2020,20200324,0063309,$0,SEMPER FI PROPERTIES,095AH01401,NaN
3,12/31/2019,20210629,0107109,$0,MILLENNIUM TRUST COMPANY LLC,095AH01401,NaN
4,12/31/2019,20210623,0105521,$0,MILLENNIUM TRUST COMPANY LLC,095AH01401,NaN
...,...,...,...,...,...,...,...
761,01/07/1998,00002286,0000265,$0,KENDRICK WILLIAM K,095AM021,NaN
762,09/25/1996,00002226,0000742,"$35,000",KENDRICK WILLIAM K,095AM021,NaN
763,02/10/1993,00002096,0001157,"$227,000",MCCORMICK MICHAEL STEPHEN,095AM021,NaN
764,07/02/1981,00001739,0000615,$0,GRIFFITH FRANCES S WRIGHT & MARGARET WRIGHT AD...,095AM021,NaN
